
## Homework

1. Complete the Python implementation of the backpropagation exercise in the **Backpropagation** section here above (cell `# try it in Python as homework!`)
    - Create the calculations for obtaining $y$ in PyTorch **using only PyTorch methods and routines**
    - Calculate the gradient
    - Check the values of the gradients and see if it is correct w.r.t. the manual calculations
2. Given the multilayer perceptron defined during the exercises from lab 1:
    - Create 10 random datapoints (with any function you wish, it can be `rand`, `randn`...) and feed them into the network
    - Given the output, calculate the Cross-Entropy loss with respect to the ground truth $[1,2,3,4,1,2,3,4,1,2]$ (classes from 1 to 4). Cross-Entropy loss:
        
        $$ CE(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{i=1}^{10} \hat{y}_i \log(y_i)$$
        
        where $y_i$ is the one-hot encoding of the $i$-th datapoint. For instance, $y_1 = [1,0,0,0]$.
        **_Note: there is an extremely handy PyTorch function for getting a one-hot encoding out of a vector, so don't try anything fancy._**
    - Backpropagate the error along the network and inspect the gradient of the parameters connecting the input layer and the first hidden layer.
3. Execute the python script `utils/randomized_backpropagation_formula.py`. This creates a formula $f(\mathbf{x})$ with randomized operators and values. Create the computational graph from this formula, do (by hand) the forward pass, then calculate (by hand) $\nabla f(\mathbf{x})$ using the backward gradient computation. Do the same calculation on PyTorch to check the correctness of your calculations. _Note: The formula created by this script is linked to your name and surname, which you have to input before_. The solution to this exercise _should_ be submitted as a scan/good quality picture of a piece of paper (or you can do it on a touch screen and submit the image...), but other formats are acceptable as well.


## Ex 1

_From the lab lesson_

**Backpropagation**

Let us suppose we have the following calculation

$\mathbf{x} = [1,~2,~-1,~3,~5]$

$ y = f(\mathbf{x}) = \log\{[\exp (x_1 * x_2 )]^2 + \sin (x_3 + x_4 + x_5) \cdot x_5\}$


Find

$\nabla f(\mathbf{x})$

In [2]:
import torch
x = torch.tensor([1.,2., -1., 3., 5.], requires_grad=True)
y = torch.log(torch.exp(x[0]*x[1])**2 + torch.sin(x[2]+x[3]+x[4])*x[4])

y.backward()

print("gradient",x.grad)
print("y", y)

gradient tensor([3.7730, 1.8865, 0.0651, 0.0651, 0.0765])
y tensor(4.0584, grad_fn=<LogBackward0>)


**Analytical calculation of the gradient in the other attachment**

## Ex 2

In [3]:
# MlpClass of the previous assignment
class MyMLP(torch.nn.Module):
    def __init__(self,bias = True):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(5,11,bias=bias),
            torch.nn.ReLU(),
            torch.nn.Linear(11,16,bias=bias),
            torch.nn.ReLU(),
            torch.nn.Linear(16,13,bias=bias),
            torch.nn.ReLU(),
            torch.nn.Linear(13,8,bias=bias),
            torch.nn.ReLU(),
            torch.nn.Linear(8,4,bias=bias),
            torch.nn.Softmax(dim=1)
        )
    def forward(self,x):
        return self.layers.forward(x)

In [4]:
from torch.nn.functional import one_hot
n = 10
x = torch.rand([n,5])
y = one_hot(torch.torch.randint(4,[n]))

model = MyMLP(bias = False)

In [5]:
def CrossEntropyLoss(y_model : torch.tensor, y_true : torch.tensor):
    assert y_model.shape[0] == y_true.shape[0], f"Mismatched number of examples got: y_model -> {y_model.shape[0]} y_true -> {y_true.shape[0]} "
    assert y_model.shape[1] == y_true.shape[1], f"Mismatched number of classes got: y_model -> {y_model.shape[1]} y_true -> {y_true.shape[1]} "

    return -torch.sum( y_true * torch.log(y_model))


In [6]:
y_model = model.forward(x)
print(y_model)

tensor([[0.2495, 0.2507, 0.2494, 0.2504],
        [0.2499, 0.2501, 0.2500, 0.2501],
        [0.2499, 0.2504, 0.2496, 0.2501],
        [0.2496, 0.2504, 0.2498, 0.2502],
        [0.2500, 0.2510, 0.2492, 0.2498],
        [0.2502, 0.2527, 0.2473, 0.2498],
        [0.2498, 0.2502, 0.2498, 0.2501],
        [0.2498, 0.2503, 0.2497, 0.2502],
        [0.2497, 0.2503, 0.2498, 0.2502],
        [0.2495, 0.2507, 0.2494, 0.2504]], grad_fn=<SoftmaxBackward0>)


In [7]:
loss = CrossEntropyLoss(y_model,y)
print(f"Loss: {loss}")

Loss: 13.872485160827637


In [8]:
#backward gradient calculation on the loss
loss.backward()


In [9]:
first_layer = list(model.parameters())[0]
print(first_layer.grad)

tensor([[-6.7753e-03, -3.5980e-03, -4.0910e-03, -5.8204e-03, -1.0078e-02],
        [ 6.0736e-04, -3.6431e-03, -4.4799e-03, -5.6679e-03, -4.6795e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.2994e-03,  6.8594e-03,  7.1887e-03, -2.1809e-03,  3.4732e-03],
        [-8.1990e-03, -1.5343e-02, -1.6551e-02, -4.9847e-03, -1.9007e-02],
        [ 9.9195e-03,  2.0639e-02,  1.9778e-02,  1.8529e-03,  2.2566e-02],
        [-5.3381e-04, -1.3853e-03, -2.4290e-03, -2.0391e-03, -8.6166e-04],
        [ 1.6323e-03,  4.2106e-04,  1.7260e-04,  3.3338e-03,  1.0164e-03],
        [ 9.8829e-03,  5.9990e-03,  6.7872e-03,  8.3505e-03,  1.4986e-02],
        [-2.7566e-05,  5.2267e-03,  3.9838e-03, -6.2747e-03, -7.2611e-05],
        [ 1.9258e-03, -2.7946e-03, -9.2548e-04,  3.4578e-04, -1.3324e-03]])


## Ex 3

Formula given by the script, input `Francesco Tomba`

$f(X) =  \tan((\text{ReLU}(x_1 * x_2) - \text{atan}(x_3 - x_4)) / x_5)$

Values given by the script: 

$\vec{X} = [ -2, 2, 3, -2, 1]$ 

**First part in the attachment**

In [15]:
x = torch.tensor([-2., 2., 3., -2., 1], requires_grad=True)
y = torch.tan((x[0]*x[1]).relu() - (x[2] - x[3]).atan() / x[4])

y.backward()
print(y)
print(x.grad)

tensor(-5.0000, grad_fn=<TanBackward0>)
tensor([ 0.0000,  0.0000, -1.0000,  1.0000, 35.7084])
